# Import Libraries

In [ ]:
import os
import sys

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot

%matplotlib inline

In [ ]:
home = os.getenv("HOME")
nlp_repo = os.path.join(home, 'git/nlp-product-sentiment-classification')
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

## Get the Data

**read Train.csv as a dataframe called train_reviews**

In [ ]:
# make path flexible
train_csv_path = os.path.join(nlp_repo, 'data/03_processed/Train.csv')
train_reviews = pd.read_csv(train_csv_path)

** Check out the info(), head(), and describe() methods on train_reviews **

# Setting up the Data
**Check train_reviews.info() again.**

In [ ]:
train_reviews.info()

## Categorical Features
Product_Type needs to be one-hot encoded

In [ ]:
train_reviews_oh = pd.get_dummies(data=train_reviews, columns=['Product_Type'])
train_reviews_oh.head()

# Text

Product_Description is the real task at hand 🤓
It needs to be tokenized and vectorized before it can be fed into a model

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

#### 2. Remove StopWords

In [ ]:
from nltk.corpus import stopwords

In [ ]:
tokens_unnested = [item for sublist in tokens for item in sublist]

In [ ]:
tokens_np = train_reviews.loc[:, 'Product_Description'].to_numpy()

In [ ]:
tokens_np[4]

In [ ]:
for index, review in enumerate(tokens_np[:16]):
    print("Index: {} Length: {}".format(index, len(review)))

In [ ]:
lengths = [len(x) for x in tokens_np]
print("min", np.min(lengths))
print("mean", np.mean(lengths))
print("std", np.std(lengths))
print("max", np.max(lengths))
plt.hist(lengths, bins=100)
plt.show()
plt.close()

In [ ]:
tokens[:2]

In [ ]:
sr = stopwords.words('english')
clean_tokens = tokens_unnested[:]

for token in tokens: 
    if token in stopwords.words('english'):
        clean_tokens.remove(token)

In [ ]:
freq = nltk.FreqDist(clean_tokens)
#for key, val in freq.items():
#    print(str(key) + ':' + str(val))
    
freq.plot(50, cumulative=False)